In [ ]:
!pip install crewai
!pip install crewai_tools
!pip install pydantic

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import GmailTool
from crewai_tools import ScrapeWebsiteTool
from crewai_tools import ScrapeElementFromWebsiteTool
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type
import os
import json

In [ ]:
website_url = 'https://torontowinterleague.tenniscores.com/?mod=nndz-TjJiOWtORzkwTlJFb0NVU1NzOD0%3D&team=nndz-WVNXOHhMOD0%3D'
club_name = "Thornhill Park Tennis Club"
date = "11/09"

In [ ]:
gc_client_id = os.getenv('gc_client_id')
gc_client_secret = os.getenv('gc_client_secret')
gc_refresh_token = os.getenv('gc_refresh_token')
openai_api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
scrape_tool = ScrapeElementFromWebsiteTool(website_url=website_url)

scrape_tool = ScrapeElementFromWebsiteTool()

scraper_agent = Agent(
    role="Web Scraper",
    goal="Extract specific information from websites",
    backstory="An expert in web scraping who can extract targeted content from web pages.",
    tools=[scrape_tool],
    verbose=True,
    llm="gpt-4o",
    openai_api_key=openai_api_key
)

scrape_task = Task(
    description="Extract row from a schedule table. Use the CSS selector '.team_schedule' to target the table and find row with date = '12/07'.",
    expected_output="A list of the main headlines from CNN.",
    agent=scraper_agent,
)

In [ ]:
class GetCapitanInput(BaseModel):
    team: str = Field(..., description="Name of the team to lookup")

class GetCapitan(BaseTool):
    name: str = "get_capitan"
    description: str = "Fetch capitans emails and names by team name from teams.json"
    args_schema: Type[BaseModel] = GetCapitanInput

    def _run(self, team: str) -> List[dict]:
        try:
            with open("teams.json", "r") as f:
                data = json.load(f)
                capitans = [entry for entry in data if entry['team'].lower() == team.lower()]
                if not capitans:
                    return [{"message": f"No capitans for for team {team}"}]
                return capitans
        except Exception as e:
            return [{"error": str(e)}]

capitan_tool = GetCapitan()

capitan_agent = Agent(
    role="Lookup email and name by team name",
    goal="Find email and name by team name",
    backstory=(
        "You can find opponet capitain email and name by team name. "
    ),
    tools=[capitan_tool],
    verbose=True,
    llm="gpt-4o",
    openai_api_key=openai_api_key
)

capitan_task = Task(
    description="Find capitains emails and name for a team by name",
    agent=capitan_agent,
)

In [ ]:
class GetMatchEmailInput(BaseModel):
    match_date: str = Field(..., description="Match date. For example: '2025-10-27'")
    match_time: str = Field(..., description="Match time. For example: '7:00pm'")

class GetMatchEmail(BaseTool):
    name:str = "get_match_email"
    description: str = "Create an email subject and body for a match at Thornhill Park Tennis Club. Input: match_date (string), optional time."
    args_schema: Type[BaseModel] = GetMatchEmailInput

    def _run(self, date: str, time: str):
        subject = f"Match {date} at {club_name}"

        body = f"""Hi Ladies,

            Our teams are scheduled to play on {date} at {time} at the {club_name}.
            The club address is 26A Old Yonge St, Thornhill, ON L4J 8C5 (at the corner of Centre St and Yonge St).
            There’s a large parking lot near the clubhouse, but you’ll need to obtain temporary parking permits. Please pick them up from the monitor inside the clubhouse and display them on your dashboards. Additional parking is available just across Old Yonge St near the Thornhill Pub.
            Looking forward to seeing your team tonight!

            Cheers,
            Julia Passynkova
        """
        return {"body": body, "subject": subject}

email_tool = GetMatchEmail()

email_agent = Agent(
    role="Tennis email writter",
    goal="Create draft email",
    backstory=(
        "You can create draft email for oppoonet team having subject and body. "
    ),
    tools=[email_tool],
    verbose=True,
    llm="gpt-4o",
    openai_api_key=openai_api_key
)

email_task = Task(
    description="Create draft reminder email for match.",
    expected_output="Create email body and subject.",
    agent=email_agent,
)

In [ ]:
gmail_tool = GmailTool(
    client_id=gc_client_id,
    client_secret=gc_client_secret,
    refresh_token=gc_refresh_token
)

gmail_agent = Agent(
    role="Tennis email sender",
    goal="Send draft email via GMail",
    backstory=(
        "You can send draft email for oppoonet emails so I can review and forward the email myself. "
    ),
    tools=[gmail_tool],
    verbose=True,
    llm="gpt-4o",
    openai_api_key=openai_api_key
)

gmail_task = Task(
    description="Draft and send a reminder email for Monday’s match.",
    expected_output="A sent email confirming match details.",
    agent=gmail_agent,
)

In [ ]:
supervisor_agent = Agent(
    role="Supervisor Agent",
    goal="Coordinate tasks among team lookup and email agents to prepare match communications.",
    backstory="Understands how to use other agents to accomplish composite goals.",
    reasoning=True,
    max_reasoning_attempts=3,
    verbose=True,
    llm="gpt-4o",
    openai_api_key=openai_api_key
)

supervisor_task = Task(
    description=(
        "Plan and coordinate the process: "
        "Find if Thornhill Park team plays at home (H) for a specific date. "
        "If Thornhill Park team plays away game (A) tell this and stop the flow. " 
        "Get opponent team name. "
        "Find opponent email and name. "
        "Create email body and subject. "
        "Send draft email for opponent emails so I can review and forward the email myself. "
    ),
    agent=supervisor_agent,
    dependencies=[scrape_task, capitan_task, email_task, gmail_task]
)

In [ ]:
crew = Crew(
  agents=[supervisor_agent],
  tasks=[supervisor_task],
  verbose=True,
  process=Process.sequential,
  #inject_date=True,  # Automatically inject current date into tasks
  #date_format="%B %d, %Y",  # Format as "May 21, 2025"
)

In [ ]:
inputs = {
    "date": date,
}

result = crew.kickoff(inputs=inputs)